# Notebook permettant d'entrainer nos agents RL

In [23]:
# Installation des paquets nécessaires
#!pip install gym numpy stable-baselines3

## Chargement de l'environnement créé

In [24]:
# Librairies
import gym
import numpy as np
from gym_env_labyrinthe import LabyrinthEnv

In [25]:
# Initialisation de l'environnement
env = LabyrinthEnv()

In [27]:
# Affichage des actions possibles et de l'espace d'observation
print("Espace des actions : ", env.action_space)
print("Espace des observations : ", env.observation_space)

Espace des actions :  Discrete(2352)
Espace des observations :  Box(0.0, 1.0, (245,), float32)


## Création d'un agent random pour tester

In [28]:
#Initialisation de l'environnement
obs = env.reset()

# Boucle de jeu avec N tours
N = 10
for i in range(N):
    action = env.action_space.sample() # Action aléatoire
    obs, recompense, fini, truncated, info = env.step(action)
    env.render()

    if fini:
        print("Partie terminée")
        obs = env.reset()

env.close()

## Entrainement de l'agent avec un algo RL

In [29]:
# Importation des librairies
from stable_baselines3 import DQN
from stable_baselines3.common.env_checker import check_env

In [30]:
#Check si l'environnement est valide
check_env(env)

Cela a permis de corriger la version de Gym utilisée. Il faut utiliser Gymnasium et plus Gym. Les fonctions ont été corrigées en conséquence dans l'environnement.

In [8]:
# Initialisation du modèle
model = DQN("MlpPolicy", 
            env, 
            learning_rate=1e-3,
            gamma=0.99,
            exploration_fraction=0.1,
            exploration_final_eps=0.02,
            verbose=1)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


c:\Users\charl\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
# Entrainement
model.learn(total_timesteps=10000)

Le joueur 1 a gagné la partie !
Le joueur 1 a gagné la partie !
Le joueur 1 a gagné la partie !
Le joueur 1 a gagné la partie !
-----------------------------------
| rollout/            |           |
|    ep_len_mean      | 930       |
|    ep_rew_mean      | -2.47e+03 |
|    exploration_rate | 0.02      |
| time/               |           |
|    episodes         | 4         |
|    fps              | 131       |
|    time_elapsed     | 28        |
|    total_timesteps  | 3722      |
| train/              |           |
|    learning_rate    | 0.001     |
|    loss             | 0.12      |
|    n_updates        | 905       |
-----------------------------------
Le joueur 1 a gagné la partie !
Le joueur 1 a gagné la partie !
Le joueur 1 a gagné la partie !
Le joueur 1 a gagné la partie !
-----------------------------------
| rollout/            |           |
|    ep_len_mean      | 1.1e+03   |
|    ep_rew_mean      | -3.78e+03 |
|    exploration_rate | 0.02      |
| time/               | 

In [10]:
# Enregistrement du modèle
model.save("modele_DQN_1")

## Evaluation du modele

In [31]:
model_charge = DQN.load("modele_DQN_1")

In [ ]:
from stable_baselines3.common.callbacks import BaseCallback
import numpy as np

class RewardLogger(BaseCallback):
    def __init__(self, verbose=0):
        super(RewardLogger, self).__init__(verbose)
        self.episode_rewards = []

    def _on_step(self) -> bool:
        # Accumule les récompenses par épisode
        if self.locals['dones']:
            episode_reward = np.sum(self.locals['rewards'])
            self.episode_rewards.append(episode_reward)
            if self.verbose > 0:
                print(f"Episode {len(self.episode_rewards)}: Reward {episode_reward}")
        return True

# Initialisation de l'environnement et du modèle
env = LabyrinthEnv()
model = DQN('MlpPolicy', env, verbose=1)

# Entrainement avec le callback
reward_logger = RewardLogger(verbose=1)
model.learn(total_timesteps=10000, callback=reward_logger)

# Affichage des récompenses accumulées après entrainement
import matplotlib.pyplot as plt
plt.plot(reward_logger.episode_rewards)
plt.title("Récompenses cumulées par épisode")
plt.xlabel("Épisodes")
plt.ylabel("Récompenses cumulées")
plt.show()
